## Project Goals ##

Github Repo: https://github.com/singhprernap/ucb_mids_207_Final_Project_Food_Deserts/tree/clyde

In [ ]:
# %pip install --quiet geopandas

# import libraries
import re
import os
import json
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import tempfile
import urllib.request
import zipfile

import geopandas as gpd
import plotly.express as px


In [ ]:
%pwd

In [ ]:
centroids_data = pd.read_csv("Data/regional_combined_centroids.csv")
centroids_data.head()

In [ ]:
# Look at shape of data
centroids_data.shape

In [ ]:
# Rename the columns
with open("Data/rename.json", 'r') as f:
    rename = json.load(f)

rename_dict = {}
for col in centroids_data.columns:
    if col in rename.keys():
         rename_dict[col] = rename[col]['simplifiedName']

centroids_data.rename(columns=rename_dict, inplace=True)

In [ ]:
centroids_data.to_excel("Data/test.xlsx", index=False)

## EDA

In [ ]:
# subset data to look at specific features
selected_centroids_data = centroids_data[[
    "TractID",
    "StateName",
    "CountyName",
    "IsUrban",
    # "Pop2010",
    "LILA_Urban1_Rural10",
    "LILA_UrbanHalf_Rural10",
    "LILA_Urban1_Rural20",
    "LILA_VehicleOr20Miles",
    "IsLowIncome",
    # "PovertyRate",
    # "MedianIncome",
    "LA_Urban1_Rural10",
    # "LAPopShare_1Mile",
    # "LALowIncomeShare_1Mile",
    # "LAPopShare_10Miles",
    # "LALowIncomeShare_10Miles",
    # "LowIncomePopulation",
    "GEOID",
    "latitude",
    "longitude",
    "state"]]

- `LAPopShare_10Miles` and `LALowIncomeShare_10Miles` have lots of missing values and may not be useful

- According to [USDA's definitions of food desert](https://extension.unr.edu/publication.aspx?PubID=2484):
    - Low Income Community:
        - This research specific: 40% population have an income <= 200% of federal poverty thresholds for family size
        - (USDA definition: poverty rate >= 20%, or a median family income <= 80% the statewide median family income)
        - IsLowIncome = 1
    - Low Access Community:
        - urban tract: >=33% population living more than 1 mile from a supermarket or grocery store
            - IsUrban = 1 and (LILA_Urban1_Rural10 = 1 or LILA_UrbanHalf_Rural10 = 1)
        - rural tract: >=33% population living more than 10 mile from a supermarket or grocery store
            - IsUrban = 0 and LILA_Urban1_Rural10 = 1
        - \>=100 households are more than 0.5 mile from a supermarket and no vehicle access,
        - or >= 500 people or 33% population live >=20% from a supermarket
            - LILA_VehicleOr20Miles = 1


In [ ]:
# Define a function to classify tracts as food deserts
def is_food_desert(row):
    is_low_income = row["IsLowIncome"] == 1
    is_low_access = False

    if row["IsUrban"] == 1:
        # Urban area: > 1 mile
        is_low_access = (
            row["LILA_Urban1_Rural10"] == 1 or
            row["LILA_UrbanHalf_Rural10"] == 1
        )
    else:
        # Rural area: > 10 miles
        is_low_access = row["LILA_Urban1_Rural10"] == 1

    # Optional vehicle or 20-mile access issue
    if row["LILA_VehicleOr20Miles"] == 1:
        is_low_access = True

    return 1 if is_low_income and is_low_access else 0


In [ ]:
# Create a new column in the DataFrame to indicate food deserts
selected_centroids_data["IsFoodDesert"] = selected_centroids_data.apply(is_food_desert, axis=1)
selected_centroids_data.head()

In [ ]:
# Check class distribution
# Extract specific colors from Set2 palette
color_dict = {0: '#71cae3',  # muted turquoise
              1: '#54a166'}  # muted green

plt.figure(figsize=(6, 4))
sns.countplot(x=selected_centroids_data["IsFoodDesert"],
              palette=color_dict,  # Use your custom color dictionary
              hue=selected_centroids_data["IsFoodDesert"]  # Use the same dataframe as x
              )  # Properly pass legend parameter

plt.title("Class Distribution of Target Variable")
plt.xlabel("Target Class")
plt.ylabel("Count")

# Save the plot to a file
# plt.savefig("/Image/class_dist_plot.png")
plt.show()

# Print class percentages
class_counts = selected_centroids_data["IsFoodDesert"].value_counts(normalize=True) * 100
print("Class Distribution (%):\n", class_counts)


In [ ]:
# Make sure the TractID column is properly converted to string type
selected_centroids_data.loc[:,'TractID'] = selected_centroids_data.loc[:,'TractID'].apply(lambda x: str(x))

In [ ]:
selected_centroids_data.info() # check that there are no null values and all datatypes are correct

In [ ]:
selected_centroids_data.describe()

In [ ]:
# Look at distribution of numeric feature vars
# Select numerical columns
numeric_cols = selected_centroids_data.select_dtypes(include=['float64', 'int64']).columns

# Plot histograms
selected_centroids_data[numeric_cols].hist(figsize=(12,10), bins=30, edgecolor='black', alpha=0.7)
plt.suptitle("Feature Distributions", fontsize=14)
plt.show()

In [ ]:
urban_counts = selected_centroids_data["IsUrban"].value_counts(normalize=True).reset_index().sort_values(by="IsUrban", ascending=True)
urban_counts.columns = ["IsUrban", "Percentage"]

# If IsUrban is 0/1, make it more readable
label_map = {0: "Rural", 1: "Urban"}
urban_counts["Label"] = urban_counts["IsUrban"].map(label_map)

# Set index to Label for easier plotting
urban_counts.set_index("Label", inplace=True)

# Plot
ax = urban_counts["Percentage"].plot(
    kind="bar",
    color=[color_dict[0], color_dict[1]],
    ylim=(0, 1),
    legend=False
)

# Title and axis formatting
plt.title("Distribution of Urban vs Rural Tracts")
plt.xlabel("Tract Type")
plt.ylabel("Percentage")
ax.yaxis.set_major_formatter(plt.matplotlib.ticker.PercentFormatter(1))

# Add text labels on top of bars
for i, pct in enumerate(urban_counts["Percentage"]):
    ax.text(
        i,
        pct + 0.02,
        f"{pct:.1%}",
        ha='center',
        va='bottom',
        fontsize=10
    )

plt.tight_layout()
plt.show()


In [ ]:
# Plot 2: IsFoodDesert percentage
fd_counts = selected_centroids_data["IsFoodDesert"].value_counts(normalize=True).reset_index()
fd_counts.columns = ["IsFoodDesert", "Percentage"]

ax = sns.barplot(
    data=fd_counts,
    x="IsFoodDesert",
    y="Percentage",
    palette=[color_dict[i] for i in fd_counts["IsFoodDesert"]]
)
plt.title("Distribution of Food Desert vs Non-Food Desert Tracts")
plt.xticks([0, 1], ["Not Food Desert", "Food Desert"])
plt.xlabel("Food Desert Status")
plt.ylabel("Percentage")
plt.ylim(0, 1)
plt.gca().yaxis.set_major_formatter(plt.matplotlib.ticker.PercentFormatter(1))

# Add text labels
for i, row in fd_counts.iterrows():
    ax.text(
        i,
        row["Percentage"] + 0.02,
        f"{row['Percentage']:.1%}",
        ha='center',
        va='bottom',
        fontsize=10
    )

plt.show()



In [ ]:
# Plot 3: Urban/Rural vs Food Desert (stacked percent bars)
cross = pd.crosstab(
    selected_centroids_data["IsUrban"],
    selected_centroids_data["IsFoodDesert"],
    normalize="index"
).reset_index().melt(
    id_vars="IsUrban",
    var_name="IsFoodDesert",
    value_name="Percentage"
)

cross


Urban area has 51% food desert. Rural Area has 24% food desert. Why? I thought rural area would have more food desert percentage.


In [ ]:
# check correlations
numeric_cols_df = selected_centroids_data[numeric_cols]

corr = numeric_cols_df.corr()["IsFoodDesert"].sort_values(ascending=False)
print("Feature Correlation with Target:\n", corr)


In [ ]:
categorical_cols = selected_centroids_data.select_dtypes(include=['object']).columns
categorical_cols


In [ ]:
plt.figure(figsize=(12, 6))
ax = sns.countplot(
    data=selected_centroids_data,
    x='StateName',
    hue='IsFoodDesert',
    palette=color_dict
)

plt.title("Distribution of StateName by Target Class")
plt.xticks(rotation=45, ha='right')
plt.ylabel("Count")
plt.xlabel("State")

# Get total counts per state
state_totals = selected_centroids_data.groupby('StateName').size().to_dict()

# Loop through each bar and annotate with percent
for p in ax.patches:
    state = p.get_x() + p.get_width() / 2
    height = p.get_height()
    
    # Get the corresponding StateName from the x location
    label = p.get_x() + p.get_width() / 2.0
    idx = int(p.get_x() + p.get_width() / 2.0)

    # x tick index corresponds to label order
    state_idx = round(p.get_x() + p.get_width() / 2.0)
    state_name = ax.get_xticklabels()[state_idx].get_text()
    
    total = state_totals.get(state_name, 1)  # fallback to 1 to avoid div-by-zero
    percent = height / total * 100
    
    if height > 0:
        ax.text(
            p.get_x() + p.get_width() / 2.,
            height + 1,
            f'{percent:.1f}%',
            ha="center",
            fontsize=9
        )

plt.tight_layout()
plt.show()


## GeoPandas Plots

In [ ]:
selected_centroids_data.columns

In [ ]:
!pwd

In [ ]:
# Load shapefile and your data
tracts = gpd.read_file("Data/cb_2020_us_tract_500k/cb_2020_us_tract_500k.shp")
merged = tracts.merge(selected_centroids_data, left_on='GEOID', right_on='TractID')
merged
# Plot
merged.plot(column='IsFoodDesert', cmap='RdYlGn_r', legend=True)


### Tract and Urban Area Distribution

In [ ]:
for c in merged.columns:
    print(c)

In [ ]:
# 1. Load shapefile (GeoDataFrame)
tracts_gdf = gpd.read_file("Data/cb_2020_us_tract_500k/cb_2020_us_tract_500k.shp")

# 2. Ensure matching keys and string types
tracts_gdf['GEOID'] = tracts_gdf['GEOID'].astype(str)
selected_centroids_data['TractID'] = selected_centroids_data['TractID'].astype(str)

# 3. Merge with additional columns (StateName, CountyName, IsUrban)
merged_gdf = tracts_gdf.merge(
    selected_centroids_data[['TractID', 'IsUrban', 'StateName', 'CountyName']],
    left_on='GEOID',
    right_on='TractID',
    how='inner'
)

# 4. Color map for urban/rural
color_map = {
    0: '#71cae3',  # rural
    1: '#54a166',  # urban
}
merged_gdf['UrbanType'] = merged_gdf['IsUrban'].map({0: 'Rural', 1: 'Urban'})

# 5. Convert CRS and to GeoJSON
merged_gdf = merged_gdf.to_crs(epsg=4326)
geojson_data = merged_gdf.__geo_interface__

# 6. Plotly choropleth with state/county in hover
fig = px.choropleth_mapbox(
    merged_gdf,
    geojson=geojson_data,
    locations=merged_gdf.index,
    color="UrbanType",
    color_discrete_map={'Rural': '#71cae3', 'Urban': '#54a166'},
    hover_data={
        "GEOID": True,
        "StateName": True,
        "CountyName": True,
        "UrbanType": True
    },
    mapbox_style="carto-positron",
    center={"lat": 37.8, "lon": -96},
    zoom=3,
    opacity=0.6
)

fig.update_layout(title="Urban vs Rural Census Tracts (with State and County Info)", margin={"r":0,"t":40,"l":0,"b":0})
fig.show()



### Food Desert Concentration by County

In [ ]:
# Load U.S. counties shapefile
counties_gdf = gpd.read_file("Data/cb_2020_us_county_20m/cb_2020_us_county_20m.shp")  # Download from Census if needed
counties_gdf['GEOID'] = counties_gdf['STATEFP'] + counties_gdf['COUNTYFP']  # Full 5-digit FIPS code

# Ensure TractID is string for matching
selected_centroids_data['TractID'] = selected_centroids_data['TractID'].astype(str)

# Create county FIPS (if not already available)
selected_centroids_data['CountyFIPS'] = selected_centroids_data['TractID'].str[:5]

# Group by county and calculate % food desert
county_stats = selected_centroids_data.groupby('CountyFIPS').agg(
    total_tracts=('TractID', 'count'),
    food_deserts=('IsFoodDesert', 'sum')
).reset_index()

county_stats['food_desert_pct'] = county_stats['food_deserts'] / county_stats['total_tracts'] * 100

# Merge with county shapefile
merged_counties = counties_gdf.merge(
    county_stats,
    left_on='GEOID',
    right_on='CountyFIPS',
    how='inner'
)

# Convert to WGS84
merged_counties = merged_counties.to_crs(epsg=4326)



In [ ]:
# Quick mapping (you can also load from a file if needed)
fips_to_state = {
    '01': 'Alabama', '02': 'Alaska', '04': 'Arizona', '05': 'Arkansas',
    '06': 'California', '08': 'Colorado', '09': 'Connecticut', '10': 'Delaware',
    '11': 'District of Columbia', '12': 'Florida', '13': 'Georgia', '15': 'Hawaii',
    '16': 'Idaho', '17': 'Illinois', '18': 'Indiana', '19': 'Iowa',
    '20': 'Kansas', '21': 'Kentucky', '22': 'Louisiana', '23': 'Maine',
    '24': 'Maryland', '25': 'Massachusetts', '26': 'Michigan', '27': 'Minnesota',
    '28': 'Mississippi', '29': 'Missouri', '30': 'Montana', '31': 'Nebraska',
    '32': 'Nevada', '33': 'New Hampshire', '34': 'New Jersey', '35': 'New Mexico',
    '36': 'New York', '37': 'North Carolina', '38': 'North Dakota', '39': 'Ohio',
    '40': 'Oklahoma', '41': 'Oregon', '42': 'Pennsylvania', '44': 'Rhode Island',
    '45': 'South Carolina', '46': 'South Dakota', '47': 'Tennessee', '48': 'Texas',
    '49': 'Utah', '50': 'Vermont', '51': 'Virginia', '53': 'Washington',
    '54': 'West Virginia', '55': 'Wisconsin', '56': 'Wyoming'
}

# Map it in your GeoDataFrame
merged_counties['StateName'] = merged_counties['STATEFP'].map(fips_to_state)


In [ ]:
fig = px.choropleth_mapbox(
    merged_counties,
    geojson=merged_counties.__geo_interface__,
    locations=merged_counties.index,
    color="food_desert_pct",
    color_continuous_scale="Reds",
    hover_data={
        "StateName": True,         # <-- new line for state name
        "NAME": True,              # County name
        "food_desert_pct": ':.1f',
        "total_tracts": True,
        "food_deserts": True
    },
    mapbox_style="carto-positron",
    center={"lat": 37.8, "lon": -96},
    zoom=3,
    opacity=0.7
)

fig.update_layout(
    title="Percentage of Food Desert Census Tracts per County",
    margin={"r":0,"t":40,"l":0,"b":0}
)
fig.show()
